In [ ]:
from qdrant_client import QdrantClient


# Initialize the client
client = QdrantClient(":memory:")


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('styles.csv', on_bad_lines='skip')

# Extract the 'productDisplayName' column into a list
product_display_names = df['productDisplayName'].astype(str).tolist()


In [ ]:
# Function to split list into batches of specified size
def batch_list(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Define batch size
batch_size = 1000

# Process the batches
for idx, batch in enumerate(batch_list(product_display_names, batch_size)):
    start_id = idx * batch_size + 1
    end_id = start_id + len(batch) - 1
    doc_ids = list(range(start_id, end_id + 1))
    client.add(
        collection_name="product_list",
        documents=batch,
        ids=doc_ids,
    )

In [ ]:
from dspy.retrieve.qdrant_rm import QdrantRM
import dspy


qdrant_retriever_model = QdrantRM("product_list", client, k=3)


ollama_model = dspy.OllamaLocal(model="llama3",model_type='text',
                                max_tokens=350,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)


dspy.settings.configure(lm= ollama_model, rm=qdrant_retriever_model)

In [54]:
class Items(dspy.Signature):
    """
    List of extracted clothing items and accessories
    """
    prompt = dspy.InputField()
    terms = dspy.OutputField(format=list)


class FindItems(dspy.Module):
    """
    Extract clothing items and accessories from a user query
    """
    def __init__(self):
        super().__init__()
        self.item_extractor = dspy.Predict(Items)

    def forward(self, question):
        instruction = (
            "Return a list of clothing items and accesores to be worn by a user, given their preferences"
            f"Preferences: {question}"
        )
        prediction = self.item_extractor(
            prompt=instruction
        )
        return prediction.terms

In [78]:
module = FindItems()

In [79]:
response = module("Tripy party with clowns and lot if music in New York")

In [80]:
print(response)

Here is the extracted list of clothing items and accessories:

**List of Clothing Items and Accessories**

* **Outfit:** A sparkly jumpsuit for dancing the night away!
	+ Top: Sequined tank top to match the party's festive vibe
	+ Bottom: Jumpsuit with glittery accents, perfect for twirling on the dance floor
	+ Shoes: High-heeled ankle boots with metallic buckles and a touch of sparkle
* **Accessories:** Add some whimsical flair!
	+ Headpiece: A colorful clown hat to make a statement (or two!)
	+ Earrings: Dangling tassels in matching colors to the jumpsuit's glittery accents
	+ Necklace: A chunky, beaded choker with a playful charm or two

**Additional Suggestions:** Consider adding some fun and quirky touches like:
* A pair of oversized sunglasses for added glamour
* A statement piece of jewelry that incorporates clowns (e.g., a clown-shaped brooch)
* Some bold, colorful nail polish to match the party's lively atmosphere!


In [46]:
trainset_dict= {
    "desert_wear": "* Lightweight, breathable long-sleeve shirt\n* Wide-brimmed hat\n* Sunglasses with UV protection\n* Cargo pants\n* Lightweight scarf or shemagh\n* Hiking boots with good ventilation\n* Moisture-wicking socks\n* Sunscreen\n* Hydration pack or water bottle\n* Lightweight jacket for cooler evenings",
    "caribbean_party": "* Flowy sundress with tropical print\n* Beach hat with colorful feathers\n* Statement necklace with seashells and beads\n* Flip flops in bright colors\n* Sunglasses with brightly colored frames\n* Lightweight, short-sleeve shirt with floral patterns\n* Comfortable shorts\n* Colorful bangles or bracelets\n* Straw tote bag\n* Lightweight scarf or wrap",
    "award_function": "* Elegant evening gown or tuxedo\n* Statement jewelry (earrings, necklace, bracelet)\n* Dress shoes or heels\n* Clutch or small evening bag\n* Cufflinks and tie (for tuxedo)\n* Evening makeup and hairstyle\n* Silk scarf or shawl\n* Formal watch\n* Perfume or cologne\n* Pocket square (for tuxedo)"
}

In [83]:
trainset = [dspy.Example(question=key, answer=value) for key, value in trainset_dict.items()]

In [84]:
from dspy import teleprompt
teleprompter = teleprompt.LabeledFewShot()
optimized_Find_Items = teleprompter.compile(student=FindItems(), trainset=trainset)

In [88]:
module = optimized_Find_Items("Belly Dancers with fire of Arabia")

In [89]:
module

"Here is the extracted list:\n\n**Clothing Items and Accessories**\n\n* Harem pants (traditional belly dance attire)\n* Midriff-baring tops (to show off moves while dancing)\n* Sparkly or beaded bras (for added flair during performances)\n* Flowing, loose-fitting shirts with intricate embroidery\n* High-waisted, flared pants for dramatic movements\n* Belly chain belts with dangling charms and pendants\n* Statement headpieces featuring tassels, fringe, or sequins\n\nThese items are inspired by the traditional attire of belly dancers from Arabia and other Middle Eastern cultures. The sparkly accessories add a touch of glamour to enhance the dancer's performance!"

In [90]:
parse_clothing_items(module)

['Harem pants (traditional belly dance attire)',
 'Midriff-baring tops (to show off moves while dancing)',
 'Sparkly or beaded bras (for added flair during performances)',
 'Flowing, loose-fitting shirts with intricate embroidery',
 'High-waisted, flared pants for dramatic movements',
 'Belly chain belts with dangling charms and pendants',
 'Statement headpieces featuring tassels, fringe, or sequins']

In [72]:
y = dspy.Retrieve()

In [74]:
type(y("shoes").passages)

list

In [76]:
def parse_clothing_items(text):
    # Split the text into lines
    lines = text.split('\n')

    # Initialize an empty list to hold the items
    items = []

    # Loop through each line
    for line in lines:
        # Check if the line starts with '*'
        if line.strip().startswith('* '):
            # Remove the '*' and any leading/trailing whitespace, then add to the list
            item = line.strip()[2:].strip()
            items.append(item)

    return items


['Waterproof jacket ($)', 'Umbrella (t)', 'Rain boots (e)', 'Water-repellent pants (r)', 'Insulated gloves (m)']
